In [0]:
df = spark.read.parquet("dbfs:/mnt/Anac/Silver/anac_silver.parquet/")

In [0]:
#1°:Selecionar somentes colunas que o cliente/setor pediu
colunas = ['Aerodromo_de_Destino', 'Aerodromo_de_Origem','Classificacao_da_Ocorrencia', 'Danos_a_Aeronave', 'Data_da_Ocorrencia','Municipio','UF','Regiao','Tipo_de_Aerodromo', 'Tipo_de_Ocorrencia','Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']
df = df.select(colunas)

In [0]:
colunas_a_somar = [
    "Lesoes_Desconhecidas_Passageiros",
    "Lesoes_Desconhecidas_Terceiros",
    "Lesoes_Desconhecidas_Tripulantes",
    "Lesoes_Fatais_Passageiros",
    "Lesoes_Fatais_Terceiros",
    "Lesoes_Fatais_Tripulantes",
    "Lesoes_Graves_Passageiros",
    "Lesoes_Graves_Terceiros",
    "Lesoes_Graves_Tripulantes",
    "Lesoes_Leves_Passageiros",
    "Lesoes_Leves_Terceiros",
    "Lesoes_Leves_Tripulantes"
]

df = df.withColumn("Total_lesoes", sum(df[somar_tudo] for somar_tudo in colunas_a_somar))

In [0]:
classificacoes_a_excluir = ["Inderteminado", "Sem registro", "Exterior"]

df = df.filter(~df["UF"].isin(classificacoes_a_excluir))

In [0]:
from pyspark.sql.functions import current_timestamp, date_format, from_utc_timestamp


df = df.withColumn("Atualizacao",\
    date_format( from_utc_timestamp( current_timestamp(), "America/Sao_Paulo"), "yyyy-MM-dd HH:mm:ss"))


In [0]:
df.write.mode("overwrite").format("parquet").partitionBy("UF").save("/mnt/Anac/Gold/anac_gold_particionado")